In [1]:
import random
import pandas as pd
import numpy as np

In [2]:
masked_df = pd.DataFrame()

In [3]:
regions = ['Cairo','Alex','Delta','Upper Egypt','Canal']
SEC = ['A','B','C1','C2','D','E']
no_of_ids = 1000

mean = 38.5
stdev = 12/3  
masked_df['ID'] = [random.randint(1000001,9999999) for _ in range(no_of_ids)]
masked_df['Age'] = [random.gauss(mean, stdev) for _ in range(no_of_ids)]
masked_df['Age'] = masked_df['Age'].astype(int)
masked_df['Gender'] = [random.choice(['Male','Female']) for _ in range(no_of_ids)]
masked_df['Region'] = random.choices(regions,weights=([20,10,40,25,5]),k=1000)
masked_df['SEC'] = random.choices(SEC,weights=([5,5,15,25,45,5]),k=1000)

In [4]:
Products = ['P' + str(x) for x in range(1,11)] 
Q1 = ['Q1' + '_' + x for x in Products]
Q2 = ['Q2' + '_' + x for x in Products]
Q3 = ['Q3' + '_' + x for x in Products]
tot_aw = ['tot_aw' + '_' + x for x in Products]
Ever_used = ['Ever_used' + '_' + x for x in Products]
P3M = ['P3M' + '_' + x for x in Products]
P1M = ['P1M' + '_' + x for x in Products]
MOU = ['MOU' + '_' + x for x in Products]

In [5]:
digits = ['0'] * (len(Products) -1) + ['1'] * 1
for i in range(0,len(masked_df)):
    random.shuffle(digits)
    masked_df.loc[i,Q1] = digits

    

C:\Users\Aly.Essam\AppData\Local\Temp\ipykernel_7532\1904907617.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  masked_df.loc[i,Q1] = digits
C:\Users\Aly.Essam\AppData\Local\Temp\ipykernel_7532\1904907617.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  masked_df.loc[i,Q1] = digits


In [6]:
def Question_generator(df,list1,list2,list3,reslist):
    for i in range(0,len(df)):
        for j in range(0,len(Products)):
            if masked_df.loc[i,list1[j]] == '1' or masked_df.loc[i,list2[j]] == '1' or masked_df.loc[i,list3[j]] == '1':
                masked_df.loc[i,reslist[j]] = '0'
            else:
                masked_df.loc[i,reslist[j]] = random.choice(['1','0'])

In [7]:
def funnel_generator(df,list1,reslist):
    for i in range(0,len(masked_df)):
        for j in range(0,len(Products)):
            if masked_df.loc[i,list1[j]] == '1':
                masked_df.loc[i,reslist[j]] = random.choice(['1','0'])
            else:
                masked_df.loc[i,reslist[j]] = '0'

In [8]:
Question_generator(masked_df,Q1,Q1,Q1,Q2)
Question_generator(masked_df,Q1,Q1,Q2,Q3)

In [9]:
for i in range(0,len(masked_df)):
    for j in range(0,len(Products)):
        if masked_df.loc[i,Q1[j]] == '1' or masked_df.loc[i,Q2[j]] == '1' or masked_df.loc[i,Q3[j]] == '1':
            masked_df.loc[i,tot_aw[j]] = '1'
        else:
            masked_df.loc[i,tot_aw[j]] = '0' 

In [10]:
funnel_generator(masked_df,tot_aw,Ever_used)
funnel_generator(masked_df,Ever_used,P3M)
funnel_generator(masked_df,P3M,P1M)


In [11]:
for i in range(0,len(masked_df)):
    P1M_list = []
    for j in range(0,len(Products)):
        if masked_df.loc[i,P1M[j]] == '1':
            P1M_list.append(j)
        else :
            masked_df.loc[i,MOU[j]] = '0'
    if (len(P1M_list) > 0):
        digits = ['0'] * (len(P1M_list) -1) + ['1'] * 1
        random.shuffle(digits)
        masked_df.loc[i,[MOU[x] for x in P1M_list]] = digits             

C:\Users\Aly.Essam\AppData\Local\Temp\ipykernel_7532\2451214564.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  masked_df.loc[i,[MOU[x] for x in P1M_list]] = digits
C:\Users\Aly.Essam\AppData\Local\Temp\ipykernel_7532\2451214564.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  masked_df.loc[i,[MOU[x] for x in P1M_list]] = digits


In [12]:
var_data = ['ID','Age','SEC','Gender','Region']

masked_df_unpivot = pd.melt(masked_df,id_vars = var_data, value_vars = masked_df.columns,   var_name = 'Question code', 
       value_name = 'value')

In [13]:
masked_df_unpivot.to_csv('final.csv',index = False)